[ ] Make NER microservice.

[ ] Make server_crypto.py and server_finance.py.

[ ] Make SA microservice.

[ ] Make server_crypto.py and server_finance.py.

[ ] Get Free Tier MongoDB Atlas.

[ ] Make script to download daily crypto chit-chat data.

References:
https://arxiv.org/pdf/1905.03423.pdf

In [1]:
import os
import grpc
import ner_pb2
import ner_pb2_grpc

ner_channel = grpc.insecure_channel(
    os.environ.get("NER_SERVER_ADDRESS")
)
ner_stub = ner_pb2_grpc.FinanceNERStub(ner_channel)

ner_response = ner_stub.ExtractEntities(
    ner_pb2.NerRequest(
        text="I love APPL stocks.",
        metadata="kek"
    )
)

ner_response

In [54]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

r = requests.get("https://coinmarketcap.com/all/views/all/")
soup = BeautifulSoup(r.content,'html.parser')
data = []

for row in soup.find_all('tr'):
    row_data = []
    for item in row.find_all('td')[:-1]:
        row_data.append(item.text)
    data.append(row_data)
data_cryptos = pd.DataFrame(data, columns = ["Rank","Name","Symbol","Market Cap","Price","Circulating Supply", "Volumne(24h)","%1h","%24h","%7d"])
data_cryptos = data_cryptos[["Name","Symbol"]].dropna()
data_cryptos

,Name,Symbol
3,BTCBitcoin,BTC
4,ETHEthereum,ETH
5,USDTTether,USDT
6,BNBBNB,BNB
7,USDCUSD Coin,USDC
8,ADACardano,ADA
9,XRPXRP,XRP
10,SOLSolana,SOL
11,LUNATerra,LUNA
12,DOTPolkadot,DOT


In [57]:
crypto_entities = []
for _,row in data_cryptos.iterrows():
    crypto_entities.append({
        "entity":row["Symbol"],
        "domain":"crypto",
        "synonims":[row["Symbol"],row["Name"][len(row["Symbol"]):]]
    })
crypto_entities

[{'entity': 'BTC', 'domain': 'crypto', 'synonims': ['BTC', 'Bitcoin']},
 {'entity': 'ETH', 'domain': 'crypto', 'synonims': ['ETH', 'Ethereum']},
 {'entity': 'USDT', 'domain': 'crypto', 'synonims': ['USDT', 'Tether']},
 {'entity': 'BNB', 'domain': 'crypto', 'synonims': ['BNB', 'BNB']},
 {'entity': 'USDC', 'domain': 'crypto', 'synonims': ['USDC', 'USD Coin']},
 {'entity': 'ADA', 'domain': 'crypto', 'synonims': ['ADA', 'Cardano']},
 {'entity': 'XRP', 'domain': 'crypto', 'synonims': ['XRP', 'XRP']},
 {'entity': 'SOL', 'domain': 'crypto', 'synonims': ['SOL', 'Solana']},
 {'entity': 'LUNA', 'domain': 'crypto', 'synonims': ['LUNA', 'Terra']},
 {'entity': 'DOT', 'domain': 'crypto', 'synonims': ['DOT', 'Polkadot']},
 {'entity': 'DOGE', 'domain': 'crypto', 'synonims': ['DOGE', 'Dogecoin']},
 {'entity': 'AVAX', 'domain': 'crypto', 'synonims': ['AVAX', 'Avalanche']},
 {'entity': 'SHIB', 'domain': 'crypto', 'synonims': ['SHIB', 'Shiba Inu']},
 {'entity': 'BUSD', 'domain': 'crypto', 'synonims': ['BU

In [5]:
import os
import sys
if ".../../ner" not in sys.path:
    sys.path.append("../../ner")

from server import FinanceNER